### Creando una Vista harcodeada

Vistas - Views
- Vistas basadas en funciones 
- Vistas basadas en clases


### Como crear una aplicacion en Django

python manage.py startapp ecommerce

funciones - nombrarlas con minusculas
clases - nombrarlas comenzando con mayusculas

### ecommerce/views.py

from django.shortcuts import render
from django.http import HttpResponse

#Create your views here
def home(request):
    return HttpResponse("<h1>Hola Mundo</h1>")

### ecommerce/urls.py

from django.urls import path
from ecommerce import views

# Create your views here.

urlpatterns = [
    path("", views.home, name="home"),
]


### Config/urls.py

from django.contrib import admin
from django.urls import include
from django.urls import path

urlpatterns = [
    path("up/", include("up.urls")),
    path("", include("pages.urls")),
    path("ecommerce/", include("ecommerce.urls")),
    path("admin/", admin.site.urls),
    path("__debug__/", include("debug_toolbar.urls")),
]


## Respuesta http y redicreccionamiento

from django.shortcuts import render
from django.http import HttpResponse, HttpResponseRedirect

from .models import ProductModel

def product_model_list_view(request):
    queryset = ProductModel.objects.all()
    print(queryset)
    return HttpResponse("Un ecommerce personalizado")

### Crud y vistas

CRUD 
- Create, Retrieve, Update, Delete(Crear, consultar, Actualizar, eliminar
- Crear modelos
- Agreagar la app a INSTALLED APPS
- Crear y aplicar migraciones
- Agregar admin

### ecommerce/Models.py

In [ ]:
from django.db import models

# Create your models here.
class ProductModel(models.Model):
    title = models.TextField()
    price = models.FloatField()

### config/settings.py

In [ ]:
INSTALLED_APPS = [
    "pages.apps.PagesConfig",
    "ecommerce.apps.EcommerceConfig",<-----------------
    "debug_toolbar",
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
    ]

### Como correr migraciones

python manage.py makemigrations

python manage.py migrate


### ecommerce/admin.py

from django.contrib import admin

from .models import ProductModel

# Register your models here.

admin.site.register(ProductModel)

### Como crear un superuser para el admin
-python manage.py createsuperuser

## Tipos basicos de Vista


- List View donde ves los usuarios creados
- Create view - donde puedes insertar los datos para crear un usuario
- retrieve and update View - si le doy click a un usuario creado y lo modifio

## ecommerce/urls

from django.urls import path

from ecommerce import views

url

### ecommerce/view.py

from django.shortcuts import render
from django.http import HttpResponse, HttpResponseRedirect

from .models import ProductModel

def product_model_list_view(request):
    queryset = ProductModel.objects.all()
    print(queryset)
    return HttpResponse("Un ecommerce personalizado")




## Usando templates

### ecommerce/views

from django.shortcuts import render
from django.http import HttpResponse, HttpResponseRedirect

from .models import ProductModel

def product_model_list_view(request):
    queryset = ProductModel.objects.all()
    print(queryset)
    template = "ecommerce/list-view.html"
    context = {
        "products": queryset
    }
    return render(request, template, context)


### Templates/ecommerce/list-view.html

<h1>
Vista de Listado
</h1>

{% for product in products %}
<li>
{{ product.title }} {{ product.price }}
</li>
{% endfor %}

## Vista de detalle

### ecommerce/views

In [ ]:
from django.contrib.auth.decorators import login_required
from django.shortcuts import render, get_object_or_404
from django.http import HttpResponse, HttpResponseRedirect

from .models import ProductModel, CategoryModel,SubcategoryModel,ReviewModel


def product_model_detail_view(request, product_id):
    instance = get_object_or_404(request,id = product_id)
    context = {
        "product": instance
    }
    template = "ecommerce/detail-view.html"
    return render(request,template,context)
# las demas view

### ecommerce/urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="list"),
    path("<int:product_id>", views.product_model_detail_view, name="detail"),
]


### templates/ecommerce/detail-view.html

In [ ]:
<h1>
    {{product.title}}
    {{product.price}}

    

</h1>


### templates/ecommerce/list-view

In [ ]:
<h1>
    Vista de Listado
    </h1>
    
    {% for product in products %}
    <li>
    <a href="/ecommerce/{{ product.id }}">{{product.title}}</a>
    </li>
    {% endfor %}

## Vista de Creacion

## ecommerce/views.py

In [ ]:
from django.contrib.auth.decorators import login_required
from django.contrib import messages
from django.shortcuts import render, get_object_or_404
from django.http import HttpResponse, HttpResponseRedirect

from .forms import ProductModelForm
from .models import ProductModel, CategoryModel,SubcategoryModel,ReviewModel

# Create your views here.

# @login required
# VISTA DE ACTUALIZACION
def product_model_update_view(request, product_id=None):
    
    instance = get_object_or_404(ProductModel,id = product_id)
    form = ProductModelForm(request.POST or None, instance = instance)
    if form.is_valid():
        instance = form.save(commit=False)
        instance.save()
        messages.success(request, "Producto actualizado con exito")
        return HttpResponseRedirect("/ecommerce/{product_id}".format(product_id=instance.id))
    context = {
        "form":form # El contexto envia el form
    }
    template = "ecommerce/update-view.html"
    return render(request, template, context)


## ecommerce/forms.py

In [ ]:
from django import forms

from .models import ProductModel

class ProductModelForm(forms.ModelForm):
    class Meta:
        model = ProductModel
        fields = [
            "title",
            "price"    
        ]

## Templates/ecommerce/create-view.html

In [ ]:
{% if messages %}
<ul class="messages">
    {% for message in messages %}
    <li{% if message.tags %} class="{{ message.tags }}"{% endif %}>{{ message }}</li>
    {% endfor %}
</ul>
{% endif %}

## templates/ecommerce/create-view.html

In [ ]:
{% include "ecommerce/messages.html" %}

<h1>
    Crear un nuevo producto
</h1>

<form method = "POST" action="."> 
{%csrf_token%}
{{ form.as_p}}

<input type= "submit" value="Crear">
</form>

## ecommerce/urls

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="list"),
    path("<int:product_id>", views.product_model_detail_view, name="detail"),
    path("create", views.product_model_create_view, name="create"),

]


## Vista Actualizacion

## ecommerce/views.py

In [ ]:
from django.contrib.auth.decorators import login_required
from django.contrib import messages
from django.shortcuts import render, get_object_or_404
from django.http import HttpResponse, HttpResponseRedirect

from .forms import ProductModelForm
from .models import ProductModel, CategoryModel,SubcategoryModel,ReviewModel

# Create your views here.

# @login required

def product_model_update_view(request, product_id=None):
    form = ProductModelForm(request.POST or None)
    instance = get_object_or_404(ProductModel,id = product_id)
    if form.is_valid():
        instance = form.save(commit=False)
        instance.save()
        messages.success(request, "Producto actualizado con exito")
        return HttpResponseRedirect("/ecommerce/{product_id}".format(product_id=instance.id))
    context = {
        "form":form # El contexto envia el form
    }
    template = "ecommerce/update-view.html"
    return render(request, template, context)


### templates/ecommerce/update-view.html

In [ ]:
{% include "ecommerce/messages.html" %}
<h1>
Actualizacion de producto {{ form.instance.title }}
</h1>    

{{  form.instance.title   }}

<form method = "POST" action="."> 
    {%csrf_token%}
    {{ form.as_p}} <!--Va a mandar un post a nuestro view y ahi se va a ejecutar la logica>Va a mandar un post a nuestro view y ahi se va a ejecutar la logica--> 
    
    <input type= "submit" value="Actualizar">
    </form>

## ecommerce/urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="list"),
    path("<int:product_id>", views.product_model_detail_view, name="detail"),
    path("create", views.product_model_create_view, name="create"),
    path("<int:product_id>/edit/", views.product_model_update_view, name="detail"),

]


## Vista para eliminar

### ecommerce/views.py

In [ ]:
from django.contrib.auth.decorators import login_required
from django.contrib import messages
from django.shortcuts import render, get_object_or_404
from django.http import HttpResponse, HttpResponseRedirect

from .forms import ProductModelForm
from .models import ProductModel, CategoryModel,SubcategoryModel,ReviewModel

#Vista de Eliminar
def product_model_delete_view(request, product_id):
    instance = get_object_or_404(ProductModel,id = product_id)
    if request.method == POST:
        instance.delete()
        HttpResponseRedirect("/ecommerce/")
        messages.success("Producto Eliminado")
    context = {
        "product": instance
    }
    template = "ecommerce/delete-view.html"
    return render(request,template,context)

### templates/ecommerce/delete-view.html

In [ ]:
{% include "ecommerce/messages.html" %}

<h1>
Eliminar {{ product.title }}
</h1>

<form method = "POST" action=""> 
{% csrf_token %}
Estas seguro que quieres eliminar el producto?

<input type= "submit" value="Eliminar">
<a href="/ecommerce/{{ product.id }}">Cancelar</a>
</form>

### ecommerce/urls.py

In [ ]:
from django.urls import path
from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="list"),
    path("<int:product_id>", views.product_model_detail_view, name="detail"),
    path("create", views.product_model_create_view, name="create"),
    path("<int:product_id>/edit/", views.product_model_update_view, name="detail"),
    path("<int:product_id>/delete/", views.product_model_delete_view, name="delete"),
]


## Busqueda en la Vista de Listado

### templates/ecommerce/search.html

In [1]:
{% include "ecommerce/messages.html" %}
<form method = "GET" action="/ecommerce/">
    <input type = "text" name = "q" placeholder = "Buscar">
</form>

SyntaxError: invalid syntax (472577232.py, line 1)

### templates/ecommerce/list-view.html

In [ ]:
{% include "ecommerce/search.html" %}
{% include "ecommerce/messages.html" %}
<h1>
    Vista de Listado
</h1>
    
    {% for product in products %}
    <li>
    <a href="/ecommerce/{{ product.id }}">{{product.title}}</a>
    </li>
    {% endfor %}

### ecommerce/views.py

In [ ]:
from django.db.models import Q

#....

# @login_required
def product_model_list_view(request):
    query = request.GET.get("q", None)
    queryset = ProductModel.objects.all()
    if query is not None:
        queryset = queryset.filter(
        Q(title_contains=query) | 
        Q(price_contains=query)
        )
    template = "ecommerce/list-view.html"
    context = {
        "products": queryset
    }
    
    if request.user._authenticated:
        template = "ecommerce/list-view.html"
    else:
        template = "ecommerce/list-view-public.html"
    
    return render(request, template, context)